## Implementation using TensorFlow

Previously having implemented this using FastAI, although not the best is still functional, I have decided to use TensorFlow to implement this again, to allow me to be more flexible and learn more about this framework. After a lot of learning I am now at a point in understanding where creating this model shouldnt be too difficult. This project will be split into 4 parts:
* Loading and preparing the data
* Creation and compilation of the model
* Training the model
* Creating the agent (this time not in the notebook!)

### Library imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_datasets as tfds

import pandas as pd
import numpy as np
import os, warnings
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib import gridspec

# matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

### Part 1 - loading and preperation of the data

In [12]:
train_ds = image_dataset_from_directory(
    'C:/Users/joshu/Desktop/osu-ai-data',
    label_mode='int',
    image_size=[60, 80],
    interpolation='nearest',
    validation_split=0.2,
    subset='training',
    batch_size=64,
    seed=42, 
)
valid_ds = image_dataset_from_directory(
    'C:/Users/joshu/Desktop/osu-ai-data',
    label_mode='int',
    image_size=[60, 80],
    interpolation='nearest',
    validation_split=0.2,
    subset='validation',
    batch_size=64,
    seed=42, 
    
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

def keypoint_labels(image, label):
    print(image)
    print(label)
    image_string = tf.io.read_file(image)
    _, x, y = file_name.stem.split('-')
    label = (int(x), int(y))
    return image, (x, y)
    

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds.take(1)
#train_ds.map(keypoint_labels)

Found 531 files belonging to 1 classes.
Using 425 files for training.
Found 531 files belonging to 1 classes.
Using 106 files for validation.


<TakeDataset element_spec=(TensorSpec(shape=(None, 60, 80, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
train = (
    train_ds.map(keypoint_labels)
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
valid = (
    valid_ds.map(keypoint_labels)
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [9]:
from os import listdir
from os.path import isfile, join

In [10]:
path = 'C:/Users/joshu/Desktop/osu-ai-data/data'

In [11]:
listdir(path)

['1-384-502.png',
 '10-149-435.png',
 '100-225-243.png',
 '101-176-372.png',
 '102-176-378.png',
 '103-176-378.png',
 '104-573-469.png',
 '105-258-323.png',
 '106-345-384.png',
 '107-228-246.png',
 '108-226-244.png',
 '109-486-287.png',
 '11-149-436.png',
 '110-586-295.png',
 '111-306-244.png',
 '112-225-244.png',
 '113-225-244.png',
 '114-225-244.png',
 '115-308-244.png',
 '116-226-244.png',
 '117-385-276.png',
 '118-225-244.png',
 '119-496-174.png',
 '12-150-437.png',
 '120-306-244.png',
 '121-574-194.png',
 '122-227-244.png',
 '123-326-238.png',
 '124-307-244.png',
 '125-308-243.png',
 '126-653-323.png',
 '127-580-199.png',
 '128-557-189.png',
 '129-574-194.png',
 '13-172-499.png',
 '130-582-115.png',
 '131-572-301.png',
 '132-650-321.png',
 '133-651-321.png',
 '134-434-273.png',
 '135-462-277.png',
 '136-402-188.png',
 '137-194-231.png',
 '138-349-130.png',
 '139-162-277.png',
 '14-137-292.png',
 '140-252-125.png',
 '141-160-277.png',
 '142-220-190.png',
 '143-395-404.png',
 '144-1

In [17]:
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

In [5]:
img = Image.open('C:/Users/joshu/Desktop/osu-ai-data/data/1-384-502.png')
np_img = np.array(img)
np_img.shape

(60, 80)

In [36]:
filenames = tf.constant(onlyfiles)
labels = []
for i in onlyfiles:
    i = i.split('.')
    _,x,y = i[0].split('-')
    labels.append((int(x),int(y)))

In [15]:
df = pd.DataFrame(columns=['x','y','img'])

In [18]:
for i in onlyfiles:
    img = Image.open(f'C:/Users/joshu/Desktop/osu-ai-data/data/{i}')
    np_img = np.array(img)
    i = i.split('.')
    _,x,y = i[0].split('-')
    df = df.append({'x': int(x), 'y': int(y), 'img': np_img}, ignore_index=True)
    # possibly normalise data by dividing each pixel by 255

In [19]:
df

,x,y,img
0,384,502,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,149,435,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,225,243,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,176,372,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,176,378,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...
526,176,378,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
527,183,397,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
528,333,362,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
529,227,245,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [22]:
!pip install -U scikit-learn

     ---------------------------------------- 8.2/8.2 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 3.7 MB/s eta 0:00:00


In [23]:
import sklearn
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [24]:
test

,x,y,img
326,419,171,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
381,250,135,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
338,339,407,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
490,514,88,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
294,413,197,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...
207,403,356,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
200,371,374,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
394,298,208,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 17..."
379,121,341,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
